In [1]:
# example of a super learner model for binary classification
from numpy import hstack
from numpy import vstack
from numpy import asarray
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
#create a list of base models
def get_models():
  models = []
  models.append(SVC(kernel='rbf',probability=True,random_state=0))
  models.append(XGBClassifier(max_depth=9, eta = 0.4, gamma = 3, n_rounds = 100))
  return models

In [3]:
# collect out of fold predictions form k-fold cross validation
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	# define split of data
	kfold = KFold(n_splits=10, shuffle=True)
	# enumerate splits
	for train_ix, test_ix in kfold.split(X):
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			# store columns
			fold_yhats.append(yhat)
		# store fold yhats as columns
		meta_X.append(hstack(fold_yhats))
	return vstack(meta_X), asarray(meta_y)

In [4]:
# fit all base models on the training dataset
def fit_base_models(X, y, models):
	for model in models:
		model.fit(X, y)

In [5]:
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model

In [6]:
# make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X)
		meta_X.append(yhat)
	meta_X = hstack(meta_X)
	# predict
	return meta_model.predict_proba(meta_X)

In [7]:
"""#Upload the dataset here
from google.colab import files
uploaded = files.upload()"""

'#Upload the dataset here\nfrom google.colab import files\nuploaded = files.upload()'

In [8]:
import io
import pandas as pd
df = pd.read_csv('X_trainData_column_modified_PZA.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [9]:
df

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,PZA
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,1,1,1,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2937,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
X = df.iloc[:,:-1].values
X.shape

(2941, 219)

In [11]:
y = df.iloc[:,-1].values
y.shape

(2941,)

In [12]:
# get models
models = get_models()
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
print('Meta ', meta_X.shape, meta_y.shape)

[16:03:25] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:31] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:36] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:42] WARNI

In [13]:
# fit base models
fit_base_models(X, y, models)

[16:04:24] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [14]:
# fit the meta model
meta_model = fit_meta_model(meta_X, meta_y)

In [15]:
"""#Upload the dataset here
from google.colab import files
uploaded = files.upload()"""

'#Upload the dataset here\nfrom google.colab import files\nuploaded = files.upload()'

In [16]:
X_test = pd.read_csv('final_X_testData_column_modified_PZA.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [17]:
import numpy as np

In [18]:
X_test

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,PZA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,?
771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?


In [19]:
X_val = X_test.iloc[:,:-1].values
X_val.shape

(773, 219)

In [20]:
yhat = super_learner_predictions(X_val, models, meta_model)


In [21]:
yhat

array([[0.03163982, 0.96836018],
       [0.89907024, 0.10092976],
       [0.41364193, 0.58635807],
       ...,
       [0.02941607, 0.97058393],
       [0.02664784, 0.97335216],
       [0.02824813, 0.97175187]])

In [22]:
submission_data = pd.read_csv('Y_testData_1_nolabels_PZA.csv')

In [23]:
ids=submission_data.iloc[:,:1].values

In [24]:
result=list()

for i in range(len(ids)):
    result.append([ ids[i,0], yhat[i,1]])

result.insert(0, ['ID','PZA'])
output = pd.DataFrame(result)
output.to_csv('PZA_results.csv',header = False, index = False)